In [4]:
pip install tensorflow

  Using cached https://files.pythonhosted.org/packages/d1/4f/950dfae467b384fc96bc6469de25d832534f6b4441033c39f914efd13418/astor-0.8.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/8a/48/a76be51647d0eb9f10e2a4511bf3ffb8cc1e6b14e9e4fab46173aa79f981/termcolor-1.1.0.tar.gz
  Using cached https://files.pythonhosted.org/packages/4e/35/11749bf99b2d4e3cceb4d55ca22590b0d7c2c62b9de38ac4a4a7f4687421/gast-0.2.2.tar.gz
  Using cached https://files.pythonhosted.org/packages/c0/4e/fd492e91abdc2d2fcb70ef453064d980688762079397f779758e055f6575/Markdown-3.1.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/02/e5/38518af393f7c214357079ce67a317307936896e961e35450b70fad2a9cf/rsa-4.0-py2.py3-none-any.whl
  Stored in directory: C:\Users\Faizan\AppData\Local\pip\Cache\wheels\2c\b1\94\43d03e130b929aae7ba3f8d15cbd7bc0d1cb5bb38a5c721833
  Stored in directory: C:\Users\Faizan\AppData\Local\pip\Cache\wheels\7c\06\54\bc84598ba1daf8f970247f550b175aaaee85f68b4b

tensorboard 2.0.1 has requirement setuptools>=41.0.0, but you'll have setuptools 40.8.0 which is incompatible.


In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords

from numpy import array
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import GlobalMaxPooling1D
from tensorflow.keras.layers import Embedding
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer

In [2]:
df_emad = pd.read_csv("Fully Preprocessed articles.csv")
df_emad.head()

EmptyDataError: No columns to parse from file

In [3]:
X = df_emad['all_preprocessed_articles']



In [4]:
y = df_emad['label']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
y_test2 = y_test

In [6]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [7]:
# Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1

maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [8]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open("glove.6B.100d.txt", encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

In [9]:
embedding_matrix = zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [10]:
vocab_size

329416

In [11]:
import seaborn as sn
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [12]:
from tensorflow.keras.utils import to_categorical

In [13]:
num_classes = 3
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)
y_test

array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

In [14]:
model = Sequential()
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False)
model.add(embedding_layer)

model.add(Flatten())
model.add(Dense(num_classes, activation='sigmoid'))

In [15]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])


print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 100)          32941600  
_________________________________________________________________
flatten (Flatten)            (None, 10000)             0         
_________________________________________________________________
dense (Dense)                (None, 3)                 30003     
Total params: 32,971,603
Trainable params: 30,003
Non-trainable params: 32,941,600
_________________________________________________________________
None


In [16]:
history = model.fit(X_train, y_train, batch_size=128, epochs=7, verbose=3, validation_split=0.25)


Train on 8166 samples, validate on 2722 samples
Epoch 1/7
Epoch 2/7
Epoch 3/7
Epoch 4/7
Epoch 5/7
Epoch 6/7
Epoch 7/7


In [17]:
y_pred = model.predict(X_test)

In [18]:
y_pred

array([[0.05069664, 0.00416431, 0.00415576],
       [0.07142666, 0.00102687, 0.00554246],
       [0.11746651, 0.10256997, 0.00910228],
       ...,
       [0.02316073, 0.14375311, 0.01253259],
       [0.5553429 , 0.05929148, 0.05812117],
       [0.0214521 , 0.00193244, 0.0018129 ]], dtype=float32)

In [19]:
list = y_pred
#def get_index()
y_pred_class = [np.argmax(a) for a in list]
#y_pred_class 

In [20]:
from sklearn.metrics import classification_report, accuracy_score
y_test = y_test2
y_pred = y_pred_class
score = accuracy_score(y_test, y_pred)
print(classification_report(y_test, y_pred))
print(f'Accuracy: {round(score*100, 2)}%')

              precision    recall  f1-score   support

           0       0.56      0.61      0.59      2228
           1       0.51      0.51      0.51      1934
           2       0.17      0.12      0.14       505

    accuracy                           0.51      4667
   macro avg       0.41      0.41      0.41      4667
weighted avg       0.50      0.51      0.51      4667

Accuracy: 51.25%
